In [1]:
import numpy as np
import pandas as pd

In [2]:
profile1 = pd.read_csv('obsProfiles/46Density.tsv', delimiter='\t')

In [3]:
profile1.head()

,Depth [m],Density [kg/m3]
0,3.30,883.894616
1,3.85,877.359798
2,4.40,870.582950
3,4.95,903.983130
4,5.50,893.091767


In [4]:
profile2 = pd.read_csv('obsProfiles/B26_2011_pit_density_noHeader.tsv', delimiter='\t')

In [5]:
# input depth given in cm and mass in g given per cube of size = 3 *5.5 * 6 cm3 = 99 cm3 = 0.9e-5 m3
profile2.head()

,Depth [cm],Mass [g]
0,3,16
1,6,20
2,9,23
3,12,32
4,15,30


In [6]:
profile2.keys()

Index(['Depth [cm]', ' Mass [g]'], dtype='object')

In [7]:
profile2['Depth[m]'] = profile2.apply(lambda row: row['Depth [cm]']/100, axis=1)

In [8]:
profile2['Density [kg/m3]'] = profile2.apply(lambda row: row[' Mass [g]']*100/9.9, axis=1)

In [9]:
profile2 = profile2.loc[:,'Depth[m]':'Density [kg/m3]']

In [11]:
profile2.head()

,Depth[m],Density [kg/m3]
0,0.03,161.616162
1,0.06,202.020202
2,0.09,232.323232
3,0.12,323.232323
4,0.15,303.030303


In [10]:
profile2.to_csv('obsProfiles/B26_2011_converted.csv')

In [14]:
profile3 = pd.read_csv('obsProfiles/NEEM2007shallowcoreDensity.txt', delimiter='\t')

In [15]:
profile3.head()

,Depth [m],Density[kg/m3]
0,1.375,330.276709
1,1.925,342.072306
2,2.475,394.581737
3,3.025,343.213815
4,3.575,418.553433
